In [ ]:
!pip install --upgrade nbformat

In [ ]:
import streamlit as st
import duckdb
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# --- Connect to DuckDB ---
con = duckdb.connect("heymax.duckdb")

# --- Load data ---
@st.cache_data
def load_all_data():
    growth_df = con.execute("SELECT * FROM growth_metrics ORDER BY activity_month").df()
    fct_df = con.execute("SELECT * FROM fct_events").df()
    dim_df = con.execute("SELECT * FROM dim_users").df()

    growth_df["activity_month"] = pd.to_datetime(growth_df["activity_month"])
    fct_df["event_ts"] = pd.to_datetime(fct_df["event_ts"])
    growth_df.fillna(0, inplace=True)
    return growth_df, fct_df, dim_df

growth_df, fct_df, dim_df = load_all_data()

# --- Sidebar: Month Filter ---
st.sidebar.header("📅 Filter by Month")
all_months = growth_df["activity_month"].dt.strftime("%Y-%m").unique()
selected_months = st.sidebar.multiselect("Select Month(s)", all_months, default=all_months)
filtered_df = growth_df[growth_df["activity_month"].dt.strftime("%Y-%m").isin(selected_months)]
filtered_df['activity_month'] = filtered_df['activity_month'].dt.strftime('%Y-%m')


# --- Title ---
st.title("📊 Monthly Growth Metrics Dashboard")
st.markdown("Analyze user lifecycle metrics and user behavior insights.")

# --- KPIs ---
if not filtered_df.empty:
    latest = filtered_df.sort_values("activity_month").iloc[-1]
    st.subheader("📌 Key Performance Indicators")
    k1, k2, k3, k4 = st.columns(4)
    k1.metric("🟢 Active Users", int(latest["active_users"]))
    k2.metric("🧍 New Users", int(latest["new_users"]))
    k3.metric("🔁 Retention Rate", f"{latest['retention_rate']:.2%}")
    k4.metric("⚡ Quick Ratio", f"{latest['quick_ratio']:.2f}")
else:
    st.warning("No data for selected months.")

# --- Lifecycle Breakdown + Ratios Chart ---
if not filtered_df.empty:
    st.subheader("📊 User Lifecycle Metrics (Bars + Lines)")

    fig = go.Figure()
    fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=filtered_df["new_users"],
                         name="New Users", marker_color="#4CAF50"))
    fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=filtered_df["retained_users"],
                         name="Retained Users", marker_color="#2196F3"))
    fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=filtered_df["resurrected_users"],
                         name="Resurrected Users", marker_color="#FFC107"))
    fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=-filtered_df["churned_users"],
                         name="Churned Users", marker_color="#f44336"))

    fig.add_trace(go.Scatter(x=filtered_df["activity_month"], y=filtered_df["retention_rate"] * 100,
                             name="Retention Rate (%)", mode="lines+markers", yaxis="y2", line=dict(color="orange")))
    fig.add_trace(go.Scatter(x=filtered_df["activity_month"], y=filtered_df["quick_ratio"],
                             name="Quick Ratio", mode="lines+markers", yaxis="y2", line=dict(color="purple")))

    fig.update_layout(
        barmode="relative",
        xaxis_title="Month",
        yaxis=dict(title="Users (churned is negative)"),
        yaxis2=dict(title="Ratios", overlaying="y", side="right"),
        legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),
        title="User Lifecycle Summary + Growth Metrics"
    )
    st.plotly_chart(fig, use_container_width=True)

# --- Active Users ---
st.subheader("🟩 Active Users Over Time")
active_fig = px.bar(filtered_df, x="activity_month", y="active_users", title="Active Users by Month",
                    labels={"activity_month": "Month", "active_users": "Users"}, color_discrete_sequence=["#00BCD4"])
st.plotly_chart(active_fig, use_container_width=True)

# --- EDA ---
st.header("🧠 Exploratory Data Analysis (EDA)")

col1, col2 = st.columns(2)
with col1:
    st.subheader("📱 Platform Distribution")
    st.plotly_chart(px.histogram(fct_df, x="platform", color="platform", title="Platform Usage"))

with col2:
    st.subheader("🌍 Country Distribution")
    st.plotly_chart(px.histogram(fct_df, x="country", color="country", title="Country Distribution"))

col3, col4 = st.columns(2)
with col3:
    st.subheader("🎯 UTM Source Distribution")
    st.plotly_chart(px.histogram(fct_df, x="utm_source", color="utm_source", title="Top UTM Sources"))

with col4:
    st.subheader("💳 Transaction Categories")
    st.plotly_chart(px.histogram(fct_df, x="transaction_category", color="transaction_category",
                                 title="Transaction Categories"))

# --- Tables ---
with st.expander("📄 Show Raw Growth Metrics"):
    st.dataframe(filtered_df)

with st.expander("📄 Show Raw Events Data"):
    st.dataframe(fct_df.head(100))


2025-06-17 10:58:43.656 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-17 10:58:43.698 
  command:

    streamlit run /Users/sanchitgarg/Desktop/heymax_project/.venv/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-17 10:58:43.699 No runtime found, using MemoryCacheStorageManager


In [4]:
con = duckdb.connect("heymax.duckdb")

# --- Load data ---
@st.cache_data
def load_all_data():
    growth_df = con.execute("SELECT * FROM growth_metrics ORDER BY activity_month").df()
    fct_df = con.execute("SELECT * FROM fct_events").df()
    dim_df = con.execute("SELECT * FROM dim_users").df()

    growth_df["activity_month"] = pd.to_datetime(growth_df["activity_month"])
    fct_df["event_ts"] = pd.to_datetime(fct_df["event_ts"])
    growth_df.fillna(0, inplace=True)
    return growth_df, fct_df, dim_df

2025-06-17 10:59:23.731 No runtime found, using MemoryCacheStorageManager


In [18]:
all_months = growth_df["activity_month"].dt.strftime("%Y-%m").unique()
selected_months = st.sidebar.multiselect("Select Month(s)", all_months, default=all_months)
filtered_df = growth_df[growth_df["activity_month"].dt.strftime("%Y-%m").isin(selected_months)]
filtered_df['activity_month'] = filtered_df['activity_month'].dt.strftime('%Y-%m')
filtered_df.head()

,activity_month,new_users,retained_users,resurrected_users,churned_users,active_users,retention_rate,quick_ratio,month_year
0,2025-03,100.0,0.0,0.0,0.0,100.0,0.00,0.0,2025-03
1,2025-04,0.0,100.0,0.0,0.0,100.0,1.00,0.0,2025-04
2,2025-05,0.0,100.0,0.0,0.0,100.0,1.00,0.0,2025-05
3,2025-06,0.0,99.0,0.0,1.0,99.0,0.99,0.0,2025-06


In [7]:
fct_df.head()

,event_ts,activity_date,activity_month,user_id,event_type,transaction_category,miles_amount,platform,utm_source,country
0,2025-05-04 09:50:20.784808,2025-05-04,2025-05-01,u_0081,miles_earned,dining,516.0,web,organic,TH
1,2025-03-10 02:42:01.632612,2025-03-10,2025-03-01,u_0015,miles_earned,ecommerce,92.0,web,organic,SG
2,2025-04-01 14:44:25.979638,2025-04-01,2025-04-01,u_0027,miles_earned,dining,493.0,android,tiktok,SG
3,2025-03-27 22:48:44.699845,2025-03-27,2025-03-01,u_0037,share,None,NaN,android,google,TH
4,2025-05-13 03:27:18.161339,2025-05-13,2025-05-01,u_0038,miles_earned,dining,599.0,ios,tiktok,TH


In [8]:
dim_df.head()

,user_id,first_seen_ts,first_seen_date,first_seen_month
0,u_0040,2025-03-08 00:05:13.019298,2025-03-08,2025-03-01
1,u_0099,2025-03-07 22:13:55.388160,2025-03-07,2025-03-01
2,u_0030,2025-03-09 19:33:07.864454,2025-03-09,2025-03-01
3,u_0036,2025-03-08 20:09:04.273108,2025-03-08,2025-03-01
4,u_0077,2025-03-08 08:06:17.965883,2025-03-08,2025-03-01


In [17]:
import nbformat
# Extract month and year
growth_df['month_year'] = growth_df['activity_month'].dt.strftime('%Y-%m')
growth_df.head()

,activity_month,new_users,retained_users,resurrected_users,churned_users,active_users,retention_rate,quick_ratio,month_year
0,2025-03-01,100.0,0.0,0.0,0.0,100.0,0.00,0.0,2025-03
1,2025-04-01,0.0,100.0,0.0,0.0,100.0,1.00,0.0,2025-04
2,2025-05-01,0.0,100.0,0.0,0.0,100.0,1.00,0.0,2025-05
3,2025-06-01,0.0,99.0,0.0,1.0,99.0,0.99,0.0,2025-06


In [ ]:
 fig = go.Figure()
# Create a vertical bar chart
fig = px.bar(growth_df, x='month_year', y='active_users', title='Active Users by Month and Year',
             labels={'month_year': 'Month-Year', 'active_users': 'Active Users'},
             color_discrete_sequence=['#00BCD4'])

fig.show()

In [ ]:
import nbformat

print(nbformat.__version__)

In [20]:
fig = go.Figure()
fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=filtered_df["new_users"],
                        name="New Users", marker_color="#4CAF50"))
fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=filtered_df["retained_users"],
                        name="Retained Users", marker_color="#2196F3"))
fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=filtered_df["resurrected_users"],
                        name="Resurrected Users", marker_color="#FFC107"))
fig.add_trace(go.Bar(x=filtered_df["activity_month"], y=-filtered_df["churned_users"],
                        name="Churned Users", marker_color="#f44336"))

fig.add_trace(go.Scatter(x=filtered_df["activity_month"], y=filtered_df["retention_rate"] * 100,
                            name="Retention Rate (%)", mode="lines+markers", yaxis="y2", line=dict(color="orange")))
fig.add_trace(go.Scatter(x=filtered_df["activity_month"], y=filtered_df["quick_ratio"],
                            name="Quick Ratio", mode="lines+markers", yaxis="y2", line=dict(color="purple")))

fig.update_layout(
    barmode="relative",
    xaxis_title="Month",
    yaxis=dict(title="Users (churned is negative)"),
    yaxis2=dict(title="Ratios", overlaying="y", side="right"),
    legend=dict(orientation="h", y=1.1, x=0.5, xanchor="center"),
    title="User Lifecycle Summary + Growth Metrics"
)
st.plotly_chart(fig, use_container_width=True)

DeltaGenerator()